In [1]:
import itertools
import random
import einops
from functools import partial
import numpy as np
import torch
import datasets
import os
import re
import pickle
from torch import Tensor
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from jaxtyping import Float, Int, Bool
from typing import Dict, Iterable, List, Tuple, Union, Literal, Optional
from transformer_lens import HookedTransformer
from transformer_lens.utils import (
    get_dataset,
    tokenize_and_concatenate,
    get_act_name,
    test_prompt,
    get_attention_mask,
)
from transformer_lens.hook_points import HookPoint
from tqdm.notebook import tqdm
import pandas as pd
from circuitsvis.activations import text_neuron_activations
from circuitsvis.topk_samples import topk_samples
from IPython.display import HTML, display
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils.circuit_analysis import get_logit_diff
from utils.tokenwise_ablation import (
    compute_ablation_modified_loss,
    load_directions,
    get_random_directions,
    get_zeroed_dir_vector,
    get_layerwise_token_mean_activations,
    ablation_hook_base,
    AblationHook,
    AblationHookIterator,
    get_batch_token_mean_activations,
    loss_fn,
    DEFAULT_DEVICE,
)
from utils.datasets import (
    OWTData,
    PileFullData,
    PileSplittedData,
    HFData,
    mask_positions,
    construct_exclude_list,
)
from utils.neuroscope import plot_top_onesided
from utils.store import ResultsFile, TensorBlockManager
from utils.path_patching import act_patch, Node, IterNode, IterSeqPos

from utils.visualization import get_attn_head_patterns, imshow_p, plot_attention_heads, scatter_attention_and_contribution_simple
from utils.visualization import get_attn_pattern, plot_attention

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
torch.set_grad_enabled(False)
torch.manual_seed(0)
random.seed(0)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_checkpoint = "EleutherAI/pythia-2.8b"

In [3]:
model = HookedTransformer.from_pretrained(
    model_checkpoint,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    device=device,
)
#model = model.to(device)
assert model.tokenizer is not None

Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer


### Knowledge Dataset

In [4]:
PROMPTS = [
    ("Known for being the most popular fruit in the world, the", " apple"),
    ("Known for being the most popular vegetable in the world, the", " humble"),
    ("Known for being the most popular car in the world, the", " Volkswagen"),
    ("Known for being the most popular country in the world, the", " United"),
    ("Known for being the most popular city in the world, the", " Big"),
    ("Known for being the most popular animal in the world, the", " dog"),
]

In [5]:
orig_prompts = [p[0] for p in PROMPTS]
orig_tokens = model.to_tokens(orig_prompts, prepend_bos=True)
flip_tokens = orig_tokens.roll(-1, dims=0)

In [6]:
batch_size = len(orig_prompts)
n_pairs = 1

def create_patching_dataset(prompts_and_answers):
    orig_prompts = [p[0] for p in prompts_and_answers]
    orig_tokens = model.to_tokens(orig_prompts, prepend_bos=True)
    flip_tokens = orig_tokens.roll(-1, dims=0)
    orig_answers = [p[1] for p in prompts_and_answers]
    flip_answers = orig_answers[1:] + [orig_answers[0]]

    print(orig_tokens.shape)
    print(flip_tokens.shape)
    print(orig_answers)
    print(flip_answers)

    if isinstance(orig_answers[0], List):
        answer_tokens = torch.empty(
                (batch_size, min(n_pairs, len(orig_answers[0])), 2), 
                device=device, 
                dtype=torch.long
            )
    else:
        answer_tokens = torch.empty(
                (batch_size, 1, 2), 
                device=device, 
                dtype=torch.long
            )
        
    for i in range(len(orig_prompts)):
        if isinstance(orig_answers[i], List):
            for pair_idx in range(n_pairs):
                orig_ans_tok = model.to_tokens(orig_answers[i][pair_idx], prepend_bos=False)
                flip_ans_tok = model.to_tokens(flip_answers[i][pair_idx], prepend_bos=False)
                answer_tokens[i, pair_idx, 0] = orig_ans_tok
                answer_tokens[i, pair_idx, 1] = flip_ans_tok
        else:
            orig_ans_tok = model.to_tokens(orig_answers[i], prepend_bos=False)
            flip_ans_tok = model.to_tokens(flip_answers[i], prepend_bos=False)
            answer_tokens[i, 0, 0] = orig_ans_tok
            answer_tokens[i, 0, 1] = flip_ans_tok


    
    orig_tokens = orig_tokens.to(device)
    flip_tokens = flip_tokens.to(device)

    return orig_tokens, flip_tokens, answer_tokens

orig_prompt_toks, flip_prompt_toks, answer_tokens = create_patching_dataset(PROMPTS)

torch.Size([6, 13])
torch.Size([6, 13])
[' apple', ' humble', ' Volkswagen', ' United', ' Big', ' dog']
[' humble', ' Volkswagen', ' United', ' Big', ' dog', ' apple']


#### Activation Patching

In [7]:
for i in range(0, len(orig_prompt_toks)):
    logits, _ = model.run_with_cache(orig_prompt_toks[i])
    log_diff = get_logit_diff(logits, answer_tokens[i].unsqueeze(0))
    #if log_diff < 0.1:
    print(model.to_string(orig_prompt_toks[i]))
    print(model.to_string(flip_prompt_toks[i]))
    print(model.to_str_tokens(answer_tokens[i]))
    print(log_diff, "\n")

<|endoftext|>Known for being the most popular fruit in the world, the
<|endoftext|>Known for being the most popular vegetable in the world, the
[' apple', ' humble']
tensor(0.5883, device='cuda:0') 

<|endoftext|>Known for being the most popular vegetable in the world, the
<|endoftext|>Known for being the most popular car in the world, the
[' humble', ' Volkswagen']
tensor(10.0164, device='cuda:0') 

<|endoftext|>Known for being the most popular car in the world, the
<|endoftext|>Known for being the most popular country in the world, the
[' Volkswagen', ' United']
tensor(5.9988, device='cuda:0') 

<|endoftext|>Known for being the most popular country in the world, the
<|endoftext|>Known for being the most popular city in the world, the
[' United', ' Big']
tensor(7.1206, device='cuda:0') 

<|endoftext|>Known for being the most popular city in the world, the
<|endoftext|>Known for being the most popular animal in the world, the
[' Big', ' dog']
tensor(4.4434, device='cuda:0') 

<|endofte

In [8]:
orig_logits, orig_cache = model.run_with_cache(orig_prompt_toks)
orig_logit_diff = get_logit_diff(orig_logits, answer_tokens, per_prompt=False)
orig_logit_diff

tensor(5.7401, device='cuda:0')

In [9]:
flip_logits, flip_cache = model.run_with_cache(flip_prompt_toks)
flip_logit_diff = get_logit_diff(flip_logits, answer_tokens, per_prompt=False)
flip_logit_diff

tensor(-5.3487, device='cuda:0')

In [10]:
def logit_diff_denoising(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens,
    flipped_logit_diff: float = flip_logit_diff,
    clean_logit_diff: float = orig_logit_diff,
    return_tensor: bool = False,
) -> Float[Tensor, ""]:
    '''
    Linear function of logit diff, calibrated so that it equals 0 when performance is
    same as on flipped input, and 1 when performance is same as on clean input.
    '''
    patched_logit_diff = get_logit_diff(logits, answer_tokens)
    ld = ((patched_logit_diff - flipped_logit_diff) / (clean_logit_diff  - flipped_logit_diff))
    if return_tensor:
        return ld
    else:
        return ld.item()


def logit_diff_noising(
        logits: Float[Tensor, "batch seq d_vocab"],
        clean_logit_diff: float = orig_logit_diff,
        corrupted_logit_diff: float = flip_logit_diff,
        answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens,
        return_tensor: bool = False,
    ) -> float:
        '''
        We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
        and -1 when performance has been destroyed (i.e. is same as ABC dataset).
        '''
        patched_logit_diff = get_logit_diff(logits, answer_tokens)
        ld = ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

        if return_tensor:
            return ld
        else:
            return ld.item()

In [12]:
# patching at each (layer, sequence position) for each of (resid_pre, attn_out, mlp_out) in turn

results = act_patch(
    model=model,
    orig_input=flip_prompt_toks,
    new_cache=orig_cache,
    patching_nodes=IterNode(["resid_pre", "attn_out", "mlp_out"], seq_pos="each"),
    patching_metric=logit_diff_denoising,
    verbose=True,
)
with open("results/tensors/2_8b_comma_test/content_act_patch_resid_layer_output.pkl", "wb") as f:
    pickle.dump(results, f)

  0%|          | 0/1248 [00:00<?, ?it/s]

results['resid_pre'].shape = (seq_pos=13, layer=32)
results['attn_out'].shape = (seq_pos=13, layer=32)
results['mlp_out'].shape = (seq_pos=13, layer=32)


In [13]:
with open("results/tensors/2_8b_comma_test/content_act_patch_resid_layer_output.pkl", "rb") as f:
    act_patch_resid_layer_output = pickle.load(f)

assert act_patch_resid_layer_output.keys() == {"resid_pre", "attn_out", "mlp_out"}
labels = [f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(orig_prompt_toks[0]))]
imshow_p(
    torch.stack([r.T for r in act_patch_resid_layer_output.values()]) * 100, # we transpose so layer is on the y-axis
    facet_col=0,
    facet_labels=["resid_pre", "attn_out", "mlp_out"],
    title="Patching at resid stream & layer outputs (corrupted -> clean)",
    labels={"x": "Sequence position", "y": "Layer", "color": "Logit diff variation"},
    x=labels,
    xaxis_tickangle=45,
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=1500,
    height=600,
    zmin=-50,
    zmax=50,
    margin={"r": 100, "l": 100}
)

### Deduction Dataset

In [14]:
import itertools

# Lists of nouns, categories (both upper and lower case), and adjectives with opposites
nouns = ["Mountain", "Dog", "Computer", "Flower", "Horse", "Whale", "Snake", "Apple"]
categories = ["Animals", "Plants", "Foods", "Places"]

# Adjectives with their clear opposites
adjectives = {
    "fast": "slow",
    "heavy": "light",
    "bright": "dim",
    "old": "new",
    "slow": "fast",
    "light": "heavy",
    "dim": "bright",
    "new": "old",
}

# Function to create the formatted prompt
def create_prompt(noun_upper, noun_lower, category_upper, category_lower, stated_adj, queried_adj, is_true):
    answer = " Yes" if is_true else " No"
    return (f"{noun_upper}s are {category_lower}. {category_upper} are {stated_adj}. Q: Are {noun_lower}s {queried_adj}? A:", f"{answer}")

# Generating prompts based on logical mappings
prompts = []
for noun, category, adjective in itertools.product(nouns, categories, list(adjectives.keys())):
    # True variation
    prompts.append(create_prompt(noun, noun.lower(), category, category.lower(), adjective, adjective, is_true=True))

    # Alt variation
    prompts.append(create_prompt(noun, noun.lower(), category, category.lower(), adjective, adjectives[adjective], is_true=False))

    # Alt variation
    prompts.append(create_prompt(noun, noun.lower(), category, category.lower(), adjectives[adjective], adjective, is_true=False))

    # True variation
    prompts.append(create_prompt(noun, noun.lower(), category, category.lower(), adjectives[adjective], adjectives[adjective], is_true=True))

# Optional: print the prompts
for prompt in prompts[:10]:  # Just printing the first 10 for brevity
    print(prompt)

PROMPTS = prompts

('Mountains are animals. Animals are fast. Q: Are mountains fast? A:', ' Yes')
('Mountains are animals. Animals are fast. Q: Are mountains slow? A:', ' No')
('Mountains are animals. Animals are slow. Q: Are mountains fast? A:', ' No')
('Mountains are animals. Animals are slow. Q: Are mountains slow? A:', ' Yes')
('Mountains are animals. Animals are heavy. Q: Are mountains heavy? A:', ' Yes')
('Mountains are animals. Animals are heavy. Q: Are mountains light? A:', ' No')
('Mountains are animals. Animals are light. Q: Are mountains heavy? A:', ' No')
('Mountains are animals. Animals are light. Q: Are mountains light? A:', ' Yes')
('Mountains are animals. Animals are bright. Q: Are mountains bright? A:', ' Yes')
('Mountains are animals. Animals are bright. Q: Are mountains dim? A:', ' No')


In [15]:
def flip_list(lst, n):
    # Ensure n is valid
    if n <= 0 or n > len(lst):
        return "Invalid value of n"

    flipped_list = []
    # Process the list in chunks of 2n
    for i in range(0, len(lst), 2 * n):
        chunk = lst[i:i + 2 * n]
        # If the chunk is less than 2n but at least n, flip what we can
        if n <= len(chunk) < 2 * n:
            flipped_list.extend(chunk[n:] + chunk[:n])
        # If the chunk is exactly 2n, flip the two n-length sub-chunks
        elif len(chunk) == 2 * n:
            flipped_list.extend(chunk[n:] + chunk[:n])
        # If the chunk is less than n, just add the remaining elements
        else:
            flipped_list.extend(chunk)

    return flipped_list

n = 2
flipped_list = flip_list(PROMPTS, n)
for prompt in flipped_list[:12]:  # Just printing the first 10 for brevity
    print(prompt)

('Mountains are animals. Animals are slow. Q: Are mountains fast? A:', ' No')
('Mountains are animals. Animals are slow. Q: Are mountains slow? A:', ' Yes')
('Mountains are animals. Animals are fast. Q: Are mountains fast? A:', ' Yes')
('Mountains are animals. Animals are fast. Q: Are mountains slow? A:', ' No')
('Mountains are animals. Animals are light. Q: Are mountains heavy? A:', ' No')
('Mountains are animals. Animals are light. Q: Are mountains light? A:', ' Yes')
('Mountains are animals. Animals are heavy. Q: Are mountains heavy? A:', ' Yes')
('Mountains are animals. Animals are heavy. Q: Are mountains light? A:', ' No')
('Mountains are animals. Animals are dim. Q: Are mountains bright? A:', ' No')
('Mountains are animals. Animals are dim. Q: Are mountains dim? A:', ' Yes')
('Mountains are animals. Animals are bright. Q: Are mountains bright? A:', ' Yes')
('Mountains are animals. Animals are bright. Q: Are mountains dim? A:', ' No')


In [16]:
def check_words(words: List[str], model: HookedTransformer, is_noun: bool = False):
    for word in words:
        if is_noun:
            word = word + "s"
        print(word)
        print(model.to_str_tokens(model.to_tokens(word, prepend_bos=False)))
        print(f"Sentence case length: {model.to_tokens(word, prepend_bos=False).shape[1]} Lower case length: {model.to_tokens(word.lower(), prepend_bos=False).shape[1]}")

print("\nNouns:")
check_words(nouns, model, is_noun=True)
print("\nCategories:")
check_words(categories, model)
print("\nAdjectives:")
check_words(adjectives.keys(), model)


Nouns:
Mountains
['Mount', 'ains']
Sentence case length: 2 Lower case length: 2
Dogs
['D', 'ogs']
Sentence case length: 2 Lower case length: 2
Computers
['Comput', 'ers']
Sentence case length: 2 Lower case length: 2
Flowers
['Flow', 'ers']
Sentence case length: 2 Lower case length: 2
Horses
['H', 'orses']
Sentence case length: 2 Lower case length: 2
Whales
['Wh', 'ales']
Sentence case length: 2 Lower case length: 2
Snakes
['Sn', 'akes']
Sentence case length: 2 Lower case length: 2
Apples
['App', 'les']
Sentence case length: 2 Lower case length: 2

Categories:
Animals
['An', 'imals']
Sentence case length: 2 Lower case length: 1
Plants
['Pl', 'ants']
Sentence case length: 2 Lower case length: 1
Foods
['Food', 's']
Sentence case length: 2 Lower case length: 1
Places
['Pl', 'aces']
Sentence case length: 2 Lower case length: 1

Adjectives:
fast
['fast']
Sentence case length: 1 Lower case length: 1
heavy
['heavy']
Sentence case length: 1 Lower case length: 1
bright
['bright']
Sentence case 

In [17]:
len(PROMPTS)

1024

In [18]:
orig_prompts = [p[0] for p in PROMPTS]
orig_prompt_toks = model.to_tokens(orig_prompts, prepend_bos=True)
flip_prompts = [p[0] for p in flipped_list]
flip_prompt_toks = model.to_tokens(flip_prompts, prepend_bos=True)

answer_tokens = torch.stack([model.to_tokens([PROMPTS[i][1], flipped_list[i][1]], prepend_bos=False) for i in range(len(PROMPTS))])

In [19]:
answer_tokens = answer_tokens.transpose(1, 2)
answer_tokens.shape

torch.Size([1024, 1, 2])

In [20]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")

# Tokenize the prompts and calculate lengths
tokenized_lengths = [len(tokenizer.encode(prompt + answer)) for prompt, answer in PROMPTS]

# Print the tokenized lengths
for i, length in enumerate(tokenized_lengths):
    print(f"Prompt {i+1}: {length} tokens")

    if length > 18:
        for idx, tok in enumerate(model.to_str_tokens(orig_tokens[i])):
            print(idx, tok)

Prompt 1: 18 tokens
Prompt 2: 18 tokens
Prompt 3: 18 tokens
Prompt 4: 18 tokens
Prompt 5: 18 tokens
Prompt 6: 18 tokens
Prompt 7: 18 tokens
Prompt 8: 18 tokens
Prompt 9: 18 tokens
Prompt 10: 18 tokens
Prompt 11: 18 tokens
Prompt 12: 18 tokens
Prompt 13: 18 tokens
Prompt 14: 18 tokens
Prompt 15: 18 tokens
Prompt 16: 18 tokens
Prompt 17: 18 tokens
Prompt 18: 18 tokens
Prompt 19: 18 tokens
Prompt 20: 18 tokens
Prompt 21: 18 tokens
Prompt 22: 18 tokens
Prompt 23: 18 tokens
Prompt 24: 18 tokens
Prompt 25: 18 tokens
Prompt 26: 18 tokens
Prompt 27: 18 tokens
Prompt 28: 18 tokens
Prompt 29: 18 tokens
Prompt 30: 18 tokens
Prompt 31: 18 tokens
Prompt 32: 18 tokens
Prompt 33: 18 tokens
Prompt 34: 18 tokens
Prompt 35: 18 tokens
Prompt 36: 18 tokens
Prompt 37: 18 tokens
Prompt 38: 18 tokens
Prompt 39: 18 tokens
Prompt 40: 18 tokens
Prompt 41: 18 tokens
Prompt 42: 18 tokens
Prompt 43: 18 tokens
Prompt 44: 18 tokens
Prompt 45: 18 tokens
Prompt 46: 18 tokens
Prompt 47: 18 tokens
Prompt 48: 18 tokens
P

In [21]:
# for i in range(0, len(orig_prompt_toks)):
#     logits, _ = model.run_with_cache(orig_prompt_toks[i])
#     log_diff = get_logit_diff(logits, answer_tokens[i].unsqueeze(0))
#     #if log_diff < 0.1:
#     print(model.to_string(orig_prompt_toks[i]))
#     print(model.to_string(flip_prompt_toks[i]))
#     print(model.to_str_tokens(answer_tokens[i]))
#     print(log_diff, "\n")

In [22]:
def get_logit_diff_in_batches(model, orig_prompt_toks, answer_tokens, batch_size=32):
    orig_logits = []
    for i in range(0, len(orig_prompt_toks), batch_size):
        logits, _ = model.run_with_cache(orig_prompt_toks[i:i+batch_size])
        orig_logits.append(logits)
    orig_logits = torch.cat(orig_logits)
    orig_logit_diff = get_logit_diff(orig_logits, answer_tokens, per_prompt=True)
    return orig_logit_diff

orig_logit_diff = get_logit_diff_in_batches(model, orig_prompt_toks[:64], answer_tokens[:64])
flip_logit_diff = get_logit_diff_in_batches(model, flip_prompt_toks[:64], answer_tokens[:64])

In [23]:
print(f"Mean original logit diff: {orig_logit_diff.mean()} Accuracy: {(orig_logit_diff > 0).sum()/orig_logit_diff.shape[0]}")
print(f"Mean flipped logit diff: {flip_logit_diff.mean()} Accuracy: {(flip_logit_diff > 0).sum()/flip_logit_diff.shape[0]}")

Mean original logit diff: 0.5050878524780273 Accuracy: 0.8125
Mean flipped logit diff: -0.5050878524780273 Accuracy: 0.1875


In [24]:
orig_prompt_toks_small = orig_prompt_toks[:32]
flip_prompt_toks_small = flip_prompt_toks[:32]
answer_tokens_small = answer_tokens[:32]

In [25]:
orig_logits, orig_cache = model.run_with_cache(orig_prompt_toks_small)
orig_logit_diff_all = get_logit_diff(orig_logits, answer_tokens_small, per_prompt=True)
orig_logit_diff = orig_logit_diff_all.mean()
print(f"Mean original logit diff: {orig_logit_diff} Accuracy: {(orig_logit_diff_all > 0).sum()/orig_logit_diff_all.shape[0]}")

Mean original logit diff: 0.5068720579147339 Accuracy: 0.8125


In [26]:
flip_logits, flip_cache = model.run_with_cache(flip_prompt_toks_small)
flip_logit_diff_all = get_logit_diff(flip_logits, answer_tokens_small, per_prompt=True)
flip_logit_diff = flip_logit_diff_all.mean()
print(f"Mean flipped logit diff: {flip_logit_diff} Accuracy: {(flip_logit_diff_all > 0).sum()/flip_logit_diff_all.shape[0]}")

Mean flipped logit diff: -0.5068720579147339 Accuracy: 0.1875


In [27]:
def logit_diff_denoising(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens_small,
    flipped_logit_diff: float = flip_logit_diff,
    clean_logit_diff: float = orig_logit_diff,
    return_tensor: bool = False,
) -> Float[Tensor, ""]:
    '''
    Linear function of logit diff, calibrated so that it equals 0 when performance is
    same as on flipped input, and 1 when performance is same as on clean input.
    '''
    patched_logit_diff = get_logit_diff(logits, answer_tokens)
    ld = ((patched_logit_diff - flipped_logit_diff) / (clean_logit_diff  - flipped_logit_diff))
    if return_tensor:
        return ld
    else:
        return ld.item()


def logit_diff_noising(
        logits: Float[Tensor, "batch seq d_vocab"],
        clean_logit_diff: float = orig_logit_diff,
        corrupted_logit_diff: float = flip_logit_diff,
        answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens_small,
        return_tensor: bool = False,
    ) -> float:
        '''
        We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
        and -1 when performance has been destroyed (i.e. is same as ABC dataset).
        '''
        patched_logit_diff = get_logit_diff(logits, answer_tokens)
        ld = ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

        if return_tensor:
            return ld
        else:
            return ld.item()

In [29]:
# patching at each (layer, sequence position) for each of (resid_pre, attn_out, mlp_out) in turn

results = act_patch(
    model=model,
    orig_input=flip_prompt_toks_small,
    new_cache=orig_cache,
    patching_nodes=IterNode(["resid_pre", "attn_out", "mlp_out"], seq_pos="each"),
    patching_metric=logit_diff_denoising,
    verbose=True,
)
with open("results/tensors/2_8b_deduction_test/content_act_patch_resid_layer_output.pkl", "wb") as f:
    pickle.dump(results, f)

  0%|          | 0/1728 [00:00<?, ?it/s]

results['resid_pre'].shape = (seq_pos=18, layer=32)
results['attn_out'].shape = (seq_pos=18, layer=32)
results['mlp_out'].shape = (seq_pos=18, layer=32)


In [30]:
with open("results/tensors/2_8b_deduction_test/content_act_patch_resid_layer_output.pkl", "rb") as f:
    act_patch_resid_layer_output = pickle.load(f)

assert act_patch_resid_layer_output.keys() == {"resid_pre", "attn_out", "mlp_out"}
labels = [f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(orig_prompt_toks[0]))]
imshow_p(
    torch.stack([r.T for r in act_patch_resid_layer_output.values()]) * 100, # we transpose so layer is on the y-axis
    facet_col=0,
    facet_labels=["resid_pre", "attn_out", "mlp_out"],
    title="Patching at resid stream & layer outputs (corrupted -> clean)",
    labels={"x": "Sequence position", "y": "Layer", "color": "Logit diff variation"},
    x=labels,
    xaxis_tickangle=45,
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=1500,
    height=600,
    zmin=-50,
    zmax=50,
    margin={"r": 100, "l": 100}
)